In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
%%time 
df = pd.read_csv('/content/drive/MyDrive/projects/IMDB_Sentiment_analysis/IMDB-Dataset.csv/IMDB-Dataset.csv')


CPU times: user 4.16 s, sys: 695 ms, total: 4.85 s
Wall time: 5.34 s


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'those', 'couldn', 'hadn', 'of', 'our', 'and', 'haven', "doesn't", 'should', 'yourself', "mightn't", 'my', 'yourselves', 'in', "wouldn't", 'then', 'why', 'yours', 'them', 'before', 'his', 'himself', 'from', "hadn't", "mustn't", 'by', 'only', 'theirs', 'whom', 'wasn', "you'll", 'both', 'when', 're', 'will', "hasn't", 'her', 'here', 'what', "needn't", 'as', 'does', 'same', 'any', 'ours', 'such', 'wouldn', 'itself', 'that', 'just', 'am', 'o', 'herself', 'is', 'than', 'which', "wasn't", 'having', 'd', "you're", 'did', 'me', 's', 'won', "aren't", 'your', 'been', 'they', 'would', 'don', 'doing', 'she', 'were', 'above', 'so', 'if', 'on', 'aren', 'or', "you'd", 'between', 'do', 'he', 'most', 't', 'up', 'are', 'mustn', 'themselves', "weren't", 'who', 'about', 'how', 'further', 'myself', 'y', 'had', 'shall', 'their', 'shan', 'weren', 'but', 'him', 'during', 'to', 'again', 'below', 'own', 'some', "haven't", 'while', 'a', "won't", 'might', 'off', "didn't", 'you', "she's", "shan't", 'be', 'no', 't

In [5]:
#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [6]:
%%time
pd.options.display.max_colwidth = 1000
#Data cleaning
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: user 30.9 s, sys: 146 ms, total: 31 s
Wall time: 34.6 s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* Isenção de responsabilidade: eu só assisti esse filme como um acordo condicional. E eu vejo filmes de graça. Eu não seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crítica mais curta de todos os tempos. Não vê este filme. É de longe o filme mais estúpido, lamenta, preguiçoso e inacreditavelmente UNFUNNY que eu já vi. É um desastre total. Mas como o meu ódio por este filme e por outros, se estende muito além de uma exibição, acho que vou continuar um pouco. Não conheço nenhuma das pessoas do filme além de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso não importa. Eles são todos horríveis, embora eu ache que esse seja o ponto. A edição é horrível e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrível do que eu pensava. Agora eu sei que esses filmes não devem ser sérios, mas vamos lá, é o cinema 101 que se alguém fizer um pequeno corte facial, ele...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

# **Feature Engineering**

In [7]:
#Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())   # 0 for negative and 1 for positive reviews aur  dataset is balnced with 60000 easch unit

0    60000
1    60000
Name: Label, dtype: int64


In [8]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

# **Lemmatization**

In [9]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]


**Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram**

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

**Tfidf Vectorizer**

TF-IDF (term frequency–inverse document frequency) vectorises words by taking into account the frequency of a word in a given document and the frequency between documents.

Mathematically, the importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus.

# **Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram**

In [13]:
# Downsample data to 50% of its original size
data_downsampled = data.sample(frac=0.5, random_state=42)

# Perform train/test split on downsampled data
train,test=train_test_split(data_downsampled,test_size=.3,random_state=42, shuffle=True)

# Initialize TfidfVectorizer
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=10000)

# Vectorize training and test sets
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

# Extract target variables
y_train = train['Label']
y_test = test['Label']


In [ ]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
#countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
#tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=10000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=5,max_df=0.95,max_features=10000)
#x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
#x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

y_train = train['Label']
y_test = test['Label']

# **Feature Selection with Chi squared**

In [14]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

Streaming output truncated to the last 5000 lines.
	. gratuitous
	. respective
	. ed
	. adequate
	. argento
	. amazed
	. crowd
	. epic
	. stripper
	. intriguing
	. favour
	. secondly
	. yet
	. thank
	. system
	. continues
	. career
	. alligator
	. incredibly
	. rather
	. manages
	. hysterical
	. jackman
	. woody
	. judged
	. abandoned
	. frankenstein
	. cute
	. wilson
	. strongest
	. rental
	. pretentious
	. imaginative
	. bridget
	. dressed
	. handle
	. steal
	. relentless
	. admission
	. line
	. johansson
	. caught
	. solo
	. unrelated
	. lush
	. ghost
	. credibility
	. shaking
	. annoy
	. fooled
	. balanced
	. struggle
	. apart
	. proving
	. artificial
	. spare
	. straight
	. plague
	. vulnerability
	. gorilla
	. thrown
	. mind
	. wahlberg
	. achievement
	. go
	. appears
	. calm
	. darkly
	. reasonable
	. heck
	. r
	. wayne
	. fiennes
	. absurd
	. power
	. present
	. sometimes
	. vinnie
	. god
	. jesse
	. audio
	. juvenile
	. pleasure
	. justin
	. reviewer
	. ton
	. walked
	. produc

# **Model Selection**

In [15]:
# Import prerequisite libraries
import sys
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# **Logistic Regression Model**

In [16]:
model_1 = LogisticRegression()


# **Training of Logistic Regression Model**

In [17]:
%%time
model_1.fit(x_train_tfidf, y_train)

CPU times: user 41.4 s, sys: 4.16 s, total: 45.6 s
Wall time: 23.9 s


LogisticRegression()

# **Evaluation on Test and Train dataset**

calculating and printing various evaluation metrics for a machine learning model trained on the train and test datasets. The model being evaluated is specified as model_1.

The evaluation metrics being calculated are precision, AUC (area under the curve), and F1 score. These metrics are calculated both on the training dataset (using the model_1.predict and model_1.predict_proba methods) and on the test dataset.

The precision_score function calculates the precision of the model's predictions, which is a measure of the proportion of positive predictions that are actually correct. The roc_auc_score function calculates the AUC, which is a measure of the model's ability to discriminate between positive and negative classes. The f1_score function calculates the F1 score, which is a balance between precision and recall.

The average parameter specifies the type of averaging to be used when calculating the metrics. The options are 'micro', 'macro', and 'weighted'. 'Micro' averaging calculates the metric by summing the individual true positives, false negatives, etc. of the class and then averaging across all classes. 'Macro' averaging calculates the metric for each class and then takes the mean of those values. 'Weighted' averaging calculates the metric for each class and then takes the average of those values, weighted by the number of true instances in each class.
________________________________________________________________________________

**precision_score:** This function calculates the precision of the model's predictions, which is a measure of the proportion of positive predictions that are actually correct. It takes three arguments: the true labels (y_train or y_test), the predicted labels (model_1.predict(x_train_tfidf) or model_1.predict(x_test_tfidf)), and the average parameter. The average parameter specifies the type of averaging to be used when calculating the metric. The options are 'micro', 'macro', and 'weighted'. 'Micro' averaging calculates the metric by summing the individual true positives, false negatives, etc. of the class and then averaging across all classes. 'Macro' averaging calculates the metric for each class and then takes the mean of those values. 'Weighted' averaging calculates the metric for each class and then takes the average of those values, weighted by the number of true instances in each class.

**roc_auc_score:**  This function calculates the AUC (area under the curve) of the model's predictions, which is a measure of the model's ability to discriminate between positive and negative classes. It takes three arguments: the true labels (y_train or y_test), the predicted probabilities (model_1.predict_proba(x_train_tfidf)[:,1] or model_1.predict_proba(x_test_tfidf)[:,1]), and the multi_class and average parameters. The multi_class parameter specifies the type of multi-class strategy to use. The average parameter specifies the type of averaging to be used when calculating the metric. The options are 'micro', 'macro', and 'weighted'. 'Micro' averaging calculates the metric by summing the individual true positives, false negatives, etc. of the class and then averaging across all classes. 'Macro' averaging calculates the metric for each class and then takes the mean of those values. 'Weighted' averaging calculates the metric for each class and then takes the average of those values, weighted by the number of true instances in each class.

**f1_score:** This function calculates the F1 score of the model's predictions, which is a balance between precision and recall.

In [18]:
%%time
print("Precision Score on training dateset for Logistic Regression: %s" % precision_score(y_train,model_1.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Logistic Regression: %s" % roc_auc_score(y_train,model_1.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_1 =f1_score(y_train,model_1.predict(x_train_tfidf),average="weighted")
print("F1 Score ftraining dateset for Logistic Regression: %s" % f1_score_train_1)
print("Precision Score on test for Logistic Regression: %s" % precision_score(y_test,model_1.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Logistic Regression: %s" % roc_auc_score(y_test,model_1.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_1 =f1_score(y_test,model_1.predict(x_test_tfidf),average="weighted")
print("F1 Score for Logistic Regression: %s" % f1_score_1)

Precision Score on training dateset for Logistic Regression: 0.9186190476190477
AUC Score on training dateset for Logistic Regression: 0.9738212484833104
F1 Score ftraining dateset for Logistic Regression: 0.9186187530965413
Precision Score on test for Logistic Regression: 0.8924444444444445
AUC Score on test for Logistic Regression: 0.9596462585705947
F1 Score for Logistic Regression: 0.8924466354152906
CPU times: user 4.57 s, sys: 836 ms, total: 5.4 s
Wall time: 3.62 s


# **Decision Tree Classifier**

In [19]:
model_2 = Pipeline(
    steps=[
        #("classifier", DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)),
    ("classifier", DecisionTreeClassifier())
    ]
)

# **Training of Decision Tree Classifier**

In [20]:
%%time
model_2.fit(x_train_tfidf,y_train)

CPU times: user 4min 26s, sys: 598 ms, total: 4min 26s
Wall time: 4min 33s


Pipeline(steps=[('classifier', DecisionTreeClassifier())])

# **Evaluation on test data and training data of Decision Tree Classifier**

In [21]:
%%time
print("Precision Score on training dateset for Decision Tree Classifier: %s" % precision_score(y_train,model_2.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(y_train,model_2.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_2 =f1_score(y_train,model_2.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" % f1_score_train_2)
print("Precision Score on test for Decision Tree Classifier: %s" % precision_score(y_test,model_2.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(y_test,model_2.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_2 =f1_score(y_test,model_2.predict(x_test_tfidf),average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_2)


Precision Score on training dateset for Decision Tree Classifier: 0.9999285714285714
AUC Score on training dateset for Decision Tree Classifier: 0.9999999863945572
F1 Score training dateset for Decision Tree Classifier: 0.9999285714272351
Precision Score on test for Decision Tree Classifier: 0.7130555555555556
AUC Score on test for Decision Tree Classifier: 0.7131853118997343
F1 Score for Decision Tree Classifier: 0.7130600802655824
CPU times: user 4.73 s, sys: 25.2 ms, total: 4.76 s
Wall time: 5.05 s


# **Decision Tree Classifier with max depth 11 to fix overfit**

In [22]:
model_3 = Pipeline(
    steps=[
        ("classifier", DecisionTreeClassifier( criterion='gini', max_depth=11, min_samples_split=2, min_samples_leaf=1)),
    ]
)

In [23]:
%%time
model_3.fit(x_train_tfidf,y_train)



CPU times: user 58.2 s, sys: 89.2 ms, total: 58.3 s
Wall time: 1min 1s


Pipeline(steps=[('classifier', DecisionTreeClassifier(max_depth=11))])

In [24]:
%%time
print("Precision Score on training dateset for Decision Tree Classifier: %s" % precision_score(y_train,model_3.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(y_train,model_3.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_3 =f1_score(y_train,model_3.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" % f1_score_train_3)
print("Precision Score on test for Decision Tree Classifier: %s" % precision_score(y_test,model_3.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(y_test,model_3.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_3 =f1_score(y_test,model_3.predict(x_test_tfidf),average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_3)


Precision Score on training dateset for Decision Tree Classifier: 0.7533571428571428
AUC Score on training dateset for Decision Tree Classifier: 0.8270344642461483
F1 Score training dateset for Decision Tree Classifier: 0.7498157842057465
Precision Score on test for Decision Tree Classifier: 0.7161111111111111
AUC Score on test for Decision Tree Classifier: 0.7691603911583472
F1 Score for Decision Tree Classifier: 0.7121855383526221
CPU times: user 4.48 s, sys: 21.6 ms, total: 4.5 s
Wall time: 4.89 s


# **Random Forest Classifier**

In [25]:
model_4 = Pipeline(
    steps=[
        #("classifier", RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None))
    ("classifier", RandomForestClassifier())]
)


In [26]:
%%time
model_4.fit(x_train_tfidf,y_train)

CPU times: user 2min 51s, sys: 418 ms, total: 2min 51s
Wall time: 2min 55s


Pipeline(steps=[('classifier', RandomForestClassifier())])

In [27]:
%%time
print("Precision Score on training dateset for Random Forest Classifier: %s" % precision_score(y_train,model_4.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(y_train,model_4.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,model_4.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" % f1_score_train_4)
print("Precision Score on test for Random Forest Classifier: %s" % precision_score(y_test,model_4.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(y_test,model_4.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,model_4.predict(x_test_tfidf),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_4)

Precision Score on training dateset for Random Forest Classifier: 0.9999285714285714
AUC Score on training dateset for Random Forest Classifier: 0.9999999251700653
F1 Score training dateset for Random Forest Classifier: 0.9999285714272351
Precision Score on test for Random Forest Classifier: 0.8532777777777778
AUC Score on test for Random Forest Classifier: 0.9285923219694596
F1 Score for Random Forest Classifier: 0.8532780046538129
CPU times: user 20.4 s, sys: 349 ms, total: 20.7 s
Wall time: 20.9 s


# **Ada Boost Classifier**

In [28]:
model_5 = Pipeline(
    steps=[
        ("classifier", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),
    n_estimators=100,
    learning_rate=.8)),
    ]
)


In [29]:
%%time
model_5.fit(x_train_tfidf,y_train)

CPU times: user 39min 48s, sys: 3.14 s, total: 39min 51s
Wall time: 40min 3s


Pipeline(steps=[('classifier',
                 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),
                                    learning_rate=0.8, n_estimators=100))])

In [30]:

%%time
print("Precision Score on training dateset for Ada Boost Classifier: %s" % precision_score(y_train,model_5.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Ada Boost Classifier: %s" % roc_auc_score(y_train,model_5.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_5 =f1_score(y_train,model_5.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Ada Boost Classifier: %s" % f1_score_train_5)
print("Precision Score on test for Ada Boost Classifier: %s" % precision_score(y_test,model_5.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Ada Boost Classifier: %s" % roc_auc_score(y_test,model_5.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_5 =f1_score(y_test,model_5.predict(x_test_tfidf),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_5)

Precision Score on training dateset for Ada Boost Classifier: 0.9194523809523809
AUC Score on training dateset for Ada Boost Classifier: 0.9802725491708861
F1 Score training dateset for Ada Boost Classifier: 0.9194523805414238
Precision Score on test for Ada Boost Classifier: 0.8403888888888889
AUC Score on test for Ada Boost Classifier: 0.897051027082354
F1 Score for Random Forest Classifier: 0.8403896027054252
CPU times: user 5min 20s, sys: 2.07 s, total: 5min 22s
Wall time: 5min 22s


# **Hyperparameter Tunning with Grid Search**

This code defines a function hyperparamtune that performs hyperparameter tuning for a given classifier using grid search. The function takes in the following arguments:

classifier: This is the classifier object that you want to tune the hyperparameters for.

param_grid: This is a dictionary of hyperparameters and their possible values that you want to tune. For example, if you want to tune the learning_rate and n_estimators hyperparameters of an AdaBoost classifier, you could specify a param_grid of {"learning_rate": [0.01, 0.1, 1.0], "n_estimators": [10, 50, 100]}.

metric: This is the metric that you want to use to evaluate the model's performance during hyperparameter tuning. The default value is "accuracy", but you can specify any metric that is available in sklearn.metrics, such as "f1_score" or "roc_auc".

verbose_value: This is an integer value that controls the verbosity of the grid search. A value of 0 means that no output is printed, while a value of 1 or higher will print progress messages.

cv: This is the number of cross-validation folds that you want to use for evaluating the model's performance during hyperparameter tuning. The default value is 5, which means that the data will be split into 5 folds and the model will be trained and evaluated 5 times, with each fold used as the test set once.

The function returns the trained GridSearchCV object and a dictionary of the best hyperparameters found by the grid search.





In [31]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

def hyperparamtune(classifier, param_grid,metric,verbose_value,cv):
    model=model_selection.GridSearchCV(
            estimator=classifier,
            param_grid=param_grid,
            scoring=metric,
            verbose=verbose_value,            
            cv=cv)

    model.fit(x_train_tfidf,y_train)
    print("Best Score %s" % {model.best_score_})
    print("Best hyperparameter set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")
    return model, best_parameters


# **Hyperparameter tunning of Logistic Regression**

 The hyperparameters being tuned are:

penalty: This is the type of regularization applied to the model. The possible values are "l2" (L2 regularization) and "l1" (L1 regularization).

C: This is the inverse of the regularization strength. A smaller value of C means that the model will be more strongly regularized.

tol: This is the tolerance for the optimization algorithm. A smaller value of tol means that the optimization will be more precise, but may also take longer to converge.

max_iter: This is the maximum number of iterations that the optimization algorithm will run for. A larger value of max_iter may lead to better model performance, but may also take longer to run.

The code then calls the hyperparamtune function with the logistic regression classifier and the param_gd dictionary as arguments, along with the "accuracy" metric and 5 cross-validation folds. The function will perform a grid search to find the best hyperparameters for the logistic regression classifier and return the trained GridSearchCV object and a dictionary of the best hyperparameters found. The %%time magic command will measure and print the time taken to run the cell.





In [ ]:
%%time
param_gd={"penalty":["l2","l1"],
         "C":[0.01,0.1,1.0,10],
         "tol":[0.0001,0.001,0.01],
         "max_iter":[100,200]}
model_7, best_param = hyperparamtune(LogisticRegression(),param_gd,"accuracy",10,5)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5; 1/48] START C=0.01, max_iter=100, penalty=l2, tol=0.0001...............
[CV 1/5; 1/48] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.850 total time=   8.1s
[CV 2/5; 1/48] START C=0.01, max_iter=100, penalty=l2, tol=0.0001...............


**Evaluation of FineTuned Logsitic Regression Classifier**

In [ ]:
%%time
print("Precision Score on training dateset for Finetuned Logsitic Regression Classifier: %s" % precision_score(y_train,model_7.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Finetuned Logsitic Regression Classifier: %s" % roc_auc_score(y_train,model_7.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_7 =f1_score(y_train,model_7.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Finetuned Logsitic Regression Classifier: %s" % f1_score_train_7)
print("Precision Score on test for Finetuned Logsitic Regression Classifier: %s" % precision_score(y_test,model_7.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Finetuned Logsitic Regression Classifier: %s" % roc_auc_score(y_test,model_7.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_7 =f1_score(y_test,model_7.predict(x_test_tfidf),average="weighted")
print("F1 Score for Finetuned Logsitic Regression Classifier: %s" % f1_score_7)


# **Hyperparameter Tunning for Random Forest Classifier**

In [ ]:
%%time
#Define grid of hyper parameters
param_gd={"n_estimators":[100,200,300],
         "max_depth":[11,13,17,19,23],
         "criterion":["gini","entropy"],
         "min_samples_split":[3,7,11],
         "min_samples_leaf":[3,5],
         "max_features":["sqrt", "log2"]}

model_8, best_param_8 = hyperparamtune(RandomForestClassifier(),param_gd,"accuracy",10,5)


In [ ]:
%%time
print("Precision Score on training dateset for Finetuned Random Forest Classifier: %s" % precision_score(y_train,model_8.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Finetuned Random Forest Classifier: %s" % roc_auc_score(y_train,model_8.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_8 =f1_score(y_train,model_8.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Finetuned Random Forest Classifier: %s" % f1_score_train_8)
print("Precision Score on test for Finetuned Random Forest Classifier: %s" % precision_score(y_test,model_8.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Finetuned Random Forest Classifier: %s" % roc_auc_score(y_test,model_8.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_8 =f1_score(y_test,model_8.predict(x_test_tfidf),average="weighted")
print("F1 Score for Finetuned Random Forest Classifier: %s" % f1_score_8)


This code compares the F1 scores of several trained models (model_1, model_2, model_3, model_4, model_5, model_7, and model_8) and assigns the model with the highest F1 score to the model variable. It then prints out the name of the model and its F1 score. The F1 score is a metric that combines precision and recall, and is a good measure of the overall performance of a classification model.

The code compares the F1 scores of the different models using a series of if statements, and assigns the model with the highest score to the model variable. If multiple models have the same highest F1 score, then the last model in the if statements (in this case, model_8) will be selected. If none of the models have a higher F1 score than the others, then the code will print out a message saying that no model was selected and to train again

In [ ]:
best_score = 0
model = None
if ((f1_score_1>f1_score_2) & (f1_score_1>f1_score_3) & (f1_score_1>f1_score_4) & (f1_score_1>f1_score_5)& (f1_score_1>f1_score_7)&(f1_score_1>f1_score_8)):
    model = model_1
    print("Logsitics Regression is providing best F1 score: %f" % f1_score_1)
elif((f1_score_2>f1_score_1) & (f1_score_2>f1_score_3) & (f1_score_2>f1_score_4) & (f1_score_2>f1_score_5)& (f1_score_2>f1_score_7)&(f1_score_2>f1_score_8)):
    model = model_2
    print(" Over fit Decision Tree is providing best F1 score: %f" % f1_score_2)
elif((f1_score_3>f1_score_1) & (f1_score_3>f1_score_2) & (f1_score_3>f1_score_4)&(f1_score_3>f1_score_5)& (f1_score_3>f1_score_7)&(f1_score_3>f1_score_8)):
    model = model_3
    print("Decision Tree is providing best F1 score: %f" % f1_score_3)
elif((f1_score_4>f1_score_1) & (f1_score_4>f1_score_2) & (f1_score_4>f1_score_3)&(f1_score_4>f1_score_5)& (f1_score_4>f1_score_7)&(f1_score_4>f1_score_8)):
    model = model_4
    print("Random Forest is providing best F1 score: %f" % f1_score_4)
elif((f1_score_5>f1_score_1) & (f1_score_5>f1_score_2) & (f1_score_5>f1_score_4)&(f1_score_5>f1_score_3)& (f1_score_5>f1_score_7)&(f1_score_5>f1_score_8)):
    model = model_5
    print("Adaboost Classifier is providing best F1 score: %f" % f1_score_5)
elif((f1_score_7>f1_score_1) & (f1_score_7>f1_score_2) & (f1_score_7>f1_score_4)&(f1_score_7>f1_score_3)& (f1_score_7>f1_score_5)&(f1_score_7>f1_score_8)):
    model = model_7
    print("Finetuned Logsitics Regression Classifier is providing best F1 score: %f" % f1_score_7)
elif((f1_score_8>f1_score_1) & (f1_score_8>f1_score_2) & (f1_score_8>f1_score_4)&(f1_score_8>f1_score_3)& (f1_score_8>f1_score_7)&(f1_score_8>f1_score_5)):
    model = model_8
    print("Finetuned Random Forest Classifier is providing best F1 score: %f" % f1_score_8)
else:
    print("No Model is selected, Train again")